In [1]:
# Import libraries

import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import splitfolders
from pathlib import Path
from torchvision.io import read_image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Use GPU
torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Global Variables
SEED = 309

IMG_SIZE = 360
ROT_SIZE = 90
TRAIN_PATH = './data/test'
TEST_PATH = './data/train'
BATCH = 4

LEARNING_RATE = 0.01
MOMENTUM = 0.9
N_EPOCHS = 2

KERNEL_SIZE = 3
PADDING = 0
STRIDE = 1

CHANNEL_OUT_1 = 32
CHANNEL_OUT_2 = 64
N_CLASSES = 3
N_CHANNELS = 3


In [2]:
# Functions

# function used for displaying images
# from https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    

# train a given model 
# also from  https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html, surprise
def trainModel(model, trainLoadern):
    criterionLossFunction = nn.CrossEntropyLoss() # loss function
    optimizer = optim.SGD(net.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

    for epoch in range(N_EPOCHS):  # loop over the dataset multiple times
        running_loss = 0.0
    
        for i, data in enumerate(trainLoader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs) # BUG [?]
            loss = criterionLossFunction(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0

In [3]:
# Models
# from https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

class MLP(nn.Module):
    
    def __init__(self):
        super().__init__()        
        self.fc1 = nn.Linear(388800, CHANNEL_OUT_2) #dont change this nr idk why??? but it won't work without???
        self.fc2 = nn.Linear(CHANNEL_OUT_2, CHANNEL_OUT_1)
        self.fc3 = nn.Linear(CHANNEL_OUT_1, N_CLASSES)
       
        
    def forward(self, x):
        x = F.relu(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class CNN_Net_1(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(N_CHANNELS, CHANNEL_OUT_1, KERNEL_SIZE)
        self.pool  = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(CHANNEL_OUT_1, CHANNEL_OUT_2, KERNEL_SIZE)
        
        self.fc1 = nn.Linear(495616, CHANNEL_OUT_2) #dont change this nr idk why??? but it won't work without???
        self.fc2 = nn.Linear(CHANNEL_OUT_2, CHANNEL_OUT_1)
        self.fc3 = nn.Linear(CHANNEL_OUT_1, N_CLASSES)
       
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        # x = nn.Softmax(x) # do it essentially propability between 0 and 1 for each class
        return x


net = MLP()

In [ ]:
# Perform Data Split
splitfolders.ratio('./origData', output="./data", seed=SEED, ratio=(.7, 0,0.3)) 

Copying files: 479 files [00:00, 1736.43 files/s]

In [ ]:
# Read data
trainDataTransform = transforms.Compose( [
                         transforms.Resize((IMG_SIZE,IMG_SIZE)),
                         transforms.RandomRotation(ROT_SIZE),
                         transforms.RandomHorizontalFlip(),
                         transforms.ToTensor() ])

train_folder = torchvision.datasets.ImageFolder(TRAIN_PATH, transform=trainDataTransform)
trainLoader = torch.utils.data.DataLoader(train_folder, shuffle=True, batch_size = BATCH)
classes = ('cherry', 'strawberry', 'tomato')

trainModel(net, trainLoader, optimizer, criterionLossFunction)

# save model
PATH = './MODEL_1.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
def testModel(model, images, labels):
    # have model analyse the data
    outputs = model(images)
    _, predicted = torch.max(outputs, 1)
    return predicted
    
def accuracyOfModel(model, images, labels):
    correct = 0
    total   = 0
    
    with torch.no_grad();
        for data in testLoader():
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total   += labels.size(0)
            correct +=(predicted == labels).sum().item()
    accuracy =  100 * correct // total
    print("Accuracy: ", accuracy, "%")
    return accuracy
    
def breakdownAccuracy(model, images, labels):
    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}

    with torch.no_grad():
        for data in testLader:
            images, labels = data
            outputs = model(images)
            _, predictions = torch.max(outputs, 1)
            
            # collect the correct predictions for each class
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1
    
    accuracies = []
    # print accuracy for each class
    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')
        accuracies.append(accuracy)
    
    return accuracies



In [ ]:
# Load test data & test!
# based on https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

testDataTransform  = transforms.Compose([transforms.Resize((IMG_SIZE,IMG_SIZE)),
                                         transforms.ToTensor()])
test_folder        = torchvision.datasets.ImageFolder(TEST_PATH,  transform=testDataTransform)  
testLoader         = torch.utils.data.DataLoader(test_folder,  shuffle=True, batch_size = BATCH)

dataiter           = iter(testLoader)
images, labels     = next(dataiter)

net                = MLP()
net.load_state_dict(torch.load(PATH))